In [2]:
import glob
import pandas as pd
import numpy as np

In [3]:
all_files = glob.glob('./*.csv')
len(all_files)

12

In [4]:
all_files

['./LABEVENTS.csv',
 './ADMISSIONS.csv',
 './D_LABITEMS.csv',
 './FULL_CHF.csv',
 './CHF_with_age.csv',
 './feature_importance.csv',
 './patients_labevents.csv',
 './petient_fillwith_lec.csv',
 './DIAGNOSES_ICD.csv',
 './petient_fillwith_mean.csv',
 './PATIENTS.csv',
 './D_ICD_DIAGNOSES.csv']

In [4]:
lab_event = pd.read_csv('./LABEVENTS.csv')

In [5]:
lab_items = pd.read_csv('./D_LABITEMS.csv')

In [6]:
adm = pd.read_csv('./ADMISSIONS.csv')

In [7]:
pat = pd.read_csv('./PATIENTS.csv')

In [8]:
d_diag = pd.read_csv('./D_ICD_DIAGNOSES.csv')

In [9]:
lab_event.shape

(27854055, 9)

In [345]:
lab_event.loc[pd.isnull(lab_event.VALUENUM)].groupby('VALUE').agg({'SUBJECT_ID':'count'}).sort_values('SUBJECT_ID', ascending = False).head()



,SUBJECT_ID
VALUE,
NEG,695567
ART,358198
NORMAL,182607
1+,136596
INTUBATED,131080


In [25]:
lab_event.loc[lab_event.VALUE.isin(['NEG','NEGATIVE'])].head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
70,351,3,NaN,51464,2101-10-14 05:00:00,NEG,NaN,EU/dL,NaN
72,353,3,NaN,51478,2101-10-14 05:00:00,NEG,NaN,mg/dL,NaN
73,354,3,NaN,51484,2101-10-14 05:00:00,NEG,NaN,mg/dL,NaN
74,355,3,NaN,51486,2101-10-14 05:00:00,NEG,NaN,NaN,NaN
75,356,3,NaN,51487,2101-10-14 05:00:00,NEG,NaN,NaN,NaN


In [26]:
lab_event = lab_event.iloc[:,[1,3,5,6,7]]

In [347]:
lab_item_count = lab_event.groupby('ITEMID').agg({'SUBJECT_ID':'count'}).reset_index().sort_values('SUBJECT_ID',ascending=False)
measurements = pd.merge(lab_item_count,lab_items.loc[:,['LABEL','ITEMID']], on = 'ITEMID', how = 'left').head(28).drop(columns = 'SUBJECT_ID')
lab_meas =  pd.merge(measurements,lab_event, on = 'ITEMID', how = 'left')



In [348]:
lab_meas.isna().sum()

ITEMID             0
LABEL              0
SUBJECT_ID         0
VALUE            104
VALUENUM        6443
VALUEUOM      471672
dtype: int64

In [349]:
lab_meas.dropna(subset= ['VALUE'], inplace = True)

In [350]:
lab_meas.loc[pd.isnull(lab_meas.VALUENUM)].groupby('VALUE').agg({'SUBJECT_ID':'count'}).sort_values('SUBJECT_ID', ascending = False).head()


,SUBJECT_ID
VALUE,
UNABLE TO REPORT,1350
ERROR,1315
>150,1241
<0.1,332
GREATER THAN 10,190


In [352]:
error_index = list(lab_meas.loc[pd.isnull(lab_meas.VALUENUM)][lab_meas.VALUE.str.contains('ERROR')].index)

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [353]:
lab_meas.drop(index = error_index, inplace = True)

In [354]:
less_than_value = lab_meas.loc[pd.isnull(lab_meas.VALUENUM)][lab_meas.VALUE.str.contains('LESS THAN')].VALUE.str.extract('(\d+)')


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [355]:
lab_meas.loc[lab_meas.VALUE.str.contains('LESS THAN'),
            'VALUENUM'] = lab_meas.loc[lab_meas.VALUE.str.contains('LESS THAN'),
                                        'VALUENUM'].fillna(value = less_than_value.astype(float)[0])
                                                                                                                                             

In [356]:
larger_than_value = lab_meas.loc[pd.isnull(lab_meas.VALUENUM)][lab_meas.VALUE.str.contains('GREATER THAN')].VALUE.str.extract('(\d+)')
lab_meas.loc[lab_meas.VALUE.str.contains('GREATER THAN'),
             'VALUENUM'] = lab_meas.loc[lab_meas.VALUE.str.contains('GREATER THAN'),
                                        'VALUENUM'].fillna(larger_than_value.astype(float)[0])


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [357]:
previous_value = lab_meas.loc[pd.isnull(lab_meas.VALUENUM)][lab_meas.VALUE.str.contains('PREVIOUSLY REPORTED AS')].VALUE.str.extract('(\d+)')
lab_meas.loc[lab_meas.VALUE.str.contains('PREVIOUSLY REPORTED AS'),
                                         'VALUENUM'] = lab_meas.loc[lab_meas.VALUE.str.contains('PREVIOUSLY REPORTED AS'),
                                                                                                'VALUENUM'].fillna(previous_value.astype(float)[0])


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [358]:
lagersign_value = lab_meas.loc[pd.isnull(lab_meas.VALUENUM)][lab_meas.VALUE.str.contains('>')].VALUE.str.extract('(\d+)')
lab_meas.loc[lab_meas.VALUE.str.contains('>'),'VALUENUM'] = lab_meas.loc[lab_meas.VALUE.str.contains('>'),'VALUENUM'].fillna(lagersign_value.astype(float)[0])


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [359]:
smallersign_value = lab_meas.loc[pd.isnull(lab_meas.VALUENUM)][lab_meas.VALUE.str.contains('<')].VALUE.str.extract('(\d+)')
lab_meas.loc[lab_meas.VALUE.str.contains('<'),'VALUENUM'] = lab_meas.loc[lab_meas.VALUE.str.contains('<'),'VALUENUM'].fillna(smallersign_value.astype(float)[0])


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [1]:
lab_meas.loc[lab_meas.VALUE == '150 IS HIGHEST MEASURED PTT','VALUENUM'] = lab_meas.loc[lab_meas.VALUE == '150 IS HIGHEST MEASURED PTT','VALUENUM'].fillna(150)


NameError: name 'lab_meas' is not defined

In [361]:
lab_meas.dropna(subset = ['VALUENUM'], inplace = True)

In [362]:
len(lab_meas.LABEL.unique())

28

In [150]:
lab_sum = lab_meas.groupby(['SUBJECT_ID', 'LABEL']).agg({'VALUENUM':['min', 'max', 'mean','std']})
lab_sum.columns = lab_sum.columns.droplevel()
pivot_table = pd.pivot_table(lab_sum.reset_index(), index='SUBJECT_ID', columns = ['LABEL'])
pivot_table.columns = list(map(lambda x: x[1] +'_' + x[0], zip(pivot_table.columns.get_level_values(0), pivot_table.columns.get_level_values(1))))

In [162]:
pivot_table.iloc[:,-28:] = pivot_table.iloc[:,-28:].fillna(0)

In [167]:
pivot_table_median = pivot_table.fillna(pivot_table.median())
pivot_table_mean = pivot_table.fillna(pivot_table.mean())

In [172]:
pivot_table_mean.head()

,Anion Gap_max,Base Excess_max,Bicarbonate_max,"Calcium, Total_max",Calculated Total CO2_max,Chloride_max,Creatinine_max,Glucose_max,Hematocrit_max,Hemoglobin_max,...,Platelet Count_std,Potassium_std,RDW_std,Red Blood Cells_std,Sodium_std,Urea Nitrogen_std,White Blood Cells_std,pCO2_std,pH_std,pO2_std
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,
2,18.298161,2.991154,29.437206,9.278462,29.288892,109.847365,1.90434,217.976133,49.6,16.5,...,210.010714,0.000000,10.960155,3.401184,0.000000,0.000000,15.485639,0.000000,0.000000,0.000000
3,23.000000,6.000000,27.000000,9.100000,34.000000,112.000000,3.20000,293.000000,38.4,12.8,...,85.275449,1.079544,0.814381,0.339164,3.303924,11.090376,4.030377,5.690502,0.054509,76.154137
4,24.000000,-1.000000,29.000000,11.000000,21.000000,109.000000,1.40000,515.000000,38.0,12.4,...,80.828149,0.440256,1.777757,0.358677,3.733118,5.707302,3.346439,0.707107,0.080829,22.627417
5,18.298161,2.991154,29.437206,9.278462,29.288892,109.847365,1.90434,217.976133,43.0,14.9,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,23.000000,0.000000,31.000000,10.800000,28.000000,110.000000,13.90000,214.000000,38.6,12.7,...,196.318075,0.562883,1.116018,0.433341,4.106324,26.818386,6.423870,5.263079,0.064187,20.791825


In [168]:
pivot_table_lecture = pivot_table.copy()

In [179]:

pivot_table_mean.iloc[:,28:56].head()

,Anion Gap_mean,Base Excess_mean,Bicarbonate_mean,"Calcium, Total_mean",Calculated Total CO2_mean,Chloride_mean,Creatinine_mean,Glucose_mean,Hematocrit_mean,Hemoglobin_mean,...,Platelet Count_mean,Potassium_mean,RDW_mean,Red Blood Cells_mean,Sodium_mean,Urea Nitrogen_mean,White Blood Cells_mean,pCO2_mean,pH_mean,pO2_mean
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,
2,13.889359,-0.525971,25.126772,8.533693,25.441753,104.060147,1.255981,129.464892,24.800000,8.250000,...,153.500000,4.172483,7.750000,2.405000,138.814651,23.958701,11.050000,41.721972,7.378615,152.131996
3,14.705882,-4.266667,22.085714,7.925926,19.844444,105.638889,1.647368,121.714286,30.818182,10.278125,...,260.031250,4.448837,15.353125,3.464687,138.027027,33.236842,12.584375,31.600000,7.388958,114.955556
4,16.191489,-3.000000,23.638298,9.440741,19.500000,97.479167,0.741509,146.294118,30.501961,10.085417,...,277.791667,3.635417,16.714583,3.396250,133.750000,16.754717,7.208333,27.500000,7.396667,58.000000
5,13.889359,-0.525971,25.126772,8.533693,25.441753,104.060147,1.255981,129.464892,43.000000,14.900000,...,309.000000,4.172483,16.700000,3.960000,138.814651,23.958701,13.900000,41.721972,7.378615,152.131996
6,16.406250,-4.400000,20.625000,8.748148,21.200000,104.937500,2.818750,117.250000,31.410417,10.340000,...,362.514286,4.906061,15.225714,3.529143,137.093750,38.500000,12.191429,37.200000,7.342000,127.400000


In [173]:
pivot_table_lecture.iloc[:,:28].columns

Index(['Anion Gap_max', 'Base Excess_max', 'Bicarbonate_max',
       'Calcium, Total_max', 'Calculated Total CO2_max', 'Chloride_max',
       'Creatinine_max', 'Glucose_max', 'Hematocrit_max', 'Hemoglobin_max',
       'INR(PT)_max', 'MCH_max', 'MCHC_max', 'MCV_max', 'Magnesium_max',
       'PT_max', 'PTT_max', 'Phosphate_max', 'Platelet Count_max',
       'Potassium_max', 'RDW_max', 'Red Blood Cells_max', 'Sodium_max',
       'Urea Nitrogen_max', 'White Blood Cells_max', 'pCO2_max', 'pH_max',
       'pO2_max'],
      dtype='object')

In [183]:
max_values=[16,  3,    30,   10.5,  29.28, 106,  9999, 
            140, 9999, 9999, 1.1,   33,    37,   96, 
            2.5, 13.5, 70,   4.5,   450,   5.2,  14.5, 
            9999,145, 20,   10.8,   42,    7.42, 100]

min_values=[8,  -4.356,23,   8.5,   21.66, 96,   9999,
            72,  9999, 9999, 0.8,   27,    31,   80,
            1.5, 11,   60,   2.5,   150,   3.6,  11.5,
            9999,135,  7,    4.3,   38,    7.38, 75]

mean_values=[12, -0.526,26.5,9.5,   25.44, 101,  9999,
            106, 9999, 9999, 0.95,  30,    34,   88,
            2.0, 12.25,65,   3.5,   300,   4.4,  13,
            9999,140,  13.5, 7.55,  36,    7.4,  87.5]

for i in range(0,28):
    pivot_table_lecture.iloc[:,i] =  pivot_table.iloc[:,i].fillna(max_values[i])
    pivot_table_lecture.iloc[:,(i+28)] =  pivot_table.iloc[:,(i+28)].fillna(mean_values[i])
    pivot_table_lecture.iloc[:,(i+56)] =  pivot_table.iloc[:,(i+56)].fillna(min_values[i])

In [203]:
np.where(pivot_table_lecture.isna())

(array([], dtype=int64), array([], dtype=int64))

In [188]:
adm = adm.loc[:,['SUBJECT_ID','INSURANCE','LANGUAGE','RELIGION','MARITAL_STATUS','ETHNICITY']].drop_duplicates('SUBJECT_ID', keep = 'last')

In [189]:
patient = pd.merge(pat.loc[:,['SUBJECT_ID', 'GENDER']], 
                   adm.loc[:,['SUBJECT_ID','INSURANCE','LANGUAGE','RELIGION','MARITAL_STATUS','ETHNICITY']], 
                   on = 'SUBJECT_ID', how = 'left')

In [192]:
petient_lecture = pd.merge(pivot_table_lecture, patient, on = 'SUBJECT_ID', how = 'left')
petient_mean = pd.merge(pivot_table_mean, patient, on = 'SUBJECT_ID', how = 'left')

In [195]:
for df in [petient_lecture, petient_mean]:
    df.LANGUAGE = df.LANGUAGE.fillna('NONE')
    df.loc[df.LANGUAGE.isin(['ENGL','NONE']) == False, 'LANGUAGE'] = 'OTHER'
    df.MARITAL_STATUS = df.MARITAL_STATUS.fillna('UNKNOWN')
    df.loc[df.MARITAL_STATUS == 'UNKNOWN (DEFAULT)','MARITAL_STATUS'] = 'UNKNOWN'
    df.loc[df.RELIGION.isin(['NOT SPECIFIED', 'UNOBTAINABLE']),'RELIGION'] = 'UNKNOWN'
    df.RELIGION = df.RELIGION.fillna('UNKNOWN')
    df.loc[df.RELIGION.isin(['UNKNOWN', 'CATHOLIC','PROTESTANT QUAKER','JEWISH']) == False,'RELIGION'] = 'OTHER'
    df.loc[df.ETHNICITY.str.contains('ASIAN'),'ETHNICITY'] = 'ASIAN'
    df.loc[df.ETHNICITY.str.contains('HISPANIC'),'ETHNICITY'] = 'HISPANIC/LATINO'
    df.loc[df.ETHNICITY.str.contains('WHITE'),'ETHNICITY'] = 'WHITE'
    df.loc[df.ETHNICITY.str.contains('BLACK'),'ETHNICITY'] = 'BLACK'
    df.loc[df.ETHNICITY.isin(['UNABLE TO OBTAIN','UNKNOWN/NOT SPECIFIED','PATIENT DECLINED TO ANSWER']),'ETHNICITY'] = 'UNKNOWN'
    df.loc[df.ETHNICITY.isin(['WHITE','UNKNOWN','BLACK','HISPANIC/LATINO','ASIAN']) == False,'ETHNICITY'] = 'OTHER'







In [211]:
values = [1.2, 0.6, 0.9,
          6.1, 4.7, 5.4,
          52,  45,  58.5,
          17.5,13.5,15.5,
          1.1, 0.5, 0.8,
          5.4, 4.2, 4.8,
          48,  37,  42.5,
          15.5, 12, 13.75]
for gender in ['M','F']:
    for feature in ['Creatinine_max', 'Creatinine_min', 'Creatinine_mean',
                    'Red Blood Cells_max','Red Blood Cells_min','Red Blood Cells_mean',
                    'Hematocrit_max', 'Hematocrit_min', 'Hematocrit_mean',
                    'Hemoglobin_max', 'Hemoglobin_min', 'Hemoglobin_mean']:
        k = 0        
        petient_lecture.loc[(petient_lecture.GENDER == gender)&(petient_lecture[feature] == 9999), feature] =  values[k]


In [213]:
petient_lecture.loc[(petient_lecture.GENDER == 'M')&(petient_lecture['Creatinine_min'] == 9999), 'Creatinine_min']

Series([], Name: Creatinine_min, dtype: float64)

## import age columns and imputer NAs as 0 (represents children)

In [215]:
age = pd.read_csv('./CHF_with_age.csv').loc[:,['SUBJECT_ID','AGE']].sort_values('AGE')

In [224]:
petient_mean = pd.merge(petient_mean, age, on = 'SUBJECT_ID', how  = 'left')

In [225]:
petient_mean = petient_mean.fillna(0)

In [220]:
petient_lecture = pd.merge(petient_lecture, age, on = 'SUBJECT_ID', how  = 'left')

In [223]:
petient_lecture = petient_lecture.fillna(0)

In [226]:
petient_lecture.shape

(45541, 120)

## Clean target feature

In [331]:
diag = pd.read_csv('./DIAGNOSES_ICD.csv')
#make sure nan values do not belong to CHF NOS
dia_pat = pd.merge(diag, d_diag.loc[:,['ICD9_CODE','SHORT_TITLE']], on = 'ICD9_CODE', how = 'left')
dia_pat[pd.isnull(dia_pat.SHORT_TITLE)][dia_pat.ICD9_CODE==4280]


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SHORT_TITLE


In [332]:
dia_pat = dia_pat.loc[:,['SUBJECT_ID','SHORT_TITLE']].fillna('None')
dia_pat.columns = ['SUBJECT_ID','CHF']
dia_pat.loc[dia_pat['CHF'].str.contains('CHF'), 'CHF'] = 1
dia_pat.loc[dia_pat['CHF']!= 1, 'CHF'] = 0

In [333]:
dia_pat = dia_pat.groupby('SUBJECT_ID').agg('sum').reset_index()
dia_pat.loc[dia_pat.CHF > 0, 'CHF'] = 1

In [334]:
dia_pat.CHF.value_counts()

0    36677
1     9843
Name: CHF, dtype: int64

In [335]:
petient_lecture = pd.merge(petient_lecture, dia_pat, on='SUBJECT_ID', how='left')

In [340]:
petient_lecture.CHF.value_counts()

0    35707
1     9834
Name: CHF, dtype: int64

In [341]:
petient_mean = pd.merge(petient_mean, dia_pat, on='SUBJECT_ID', how='left')

In [342]:
petient_mean.CHF.value_counts()

0    35707
1     9834
Name: CHF, dtype: int64

In [343]:
petient_mean.to_csv('petient_fillwith_mean.csv')

In [344]:
petient_lecture.to_csv('petient_fillwith_lec.csv')

In [25]:
for i in range(-6,-1):
    print(petient_final.iloc[:,i].value_counts())
    print('-'*30)


M    25972
F    20270
Name: GENDER, dtype: int64
------------------------------
Medicare      20890
Private       18965
Medicaid       4342
Government     1485
Self Pay        560
Name: INSURANCE, dtype: int64
------------------------------
ENGL     21743
NONE     21095
OTHER     3404
Name: LANGUAGE, dtype: int64
------------------------------
UNKNOWN              17519
CATHOLIC             15626
PROTESTANT QUAKER     5112
OTHER                 4180
JEWISH                3805
Name: RELIGION, dtype: int64
------------------------------
MARRIED        18473
UNKNOWN         9849
SINGLE          9749
POSTMARRIED     8171
Name: MARITAL_STATUS, dtype: int64
------------------------------
